In [1]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
import os
from skimage import io, color, transform
import numpy as np

In [2]:
def load_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            img = io.imread(os.path.join(folder_path, filename))
            img = color.rgb2gray(img)
            img = transform.resize(img, (64, 64) , anti_aliasing=True)
            images.append(img.flatten())
            labels.append(label)
    return images, labels

In [3]:
circle_images, circle_labels = load_images("dataset/train/circle", 0)
square_images, square_labels = load_images("dataset/train/square", 1)
triangle_images, triangle_labels = load_images("dataset/train/triangle", 2)

X_train = np.concatenate((circle_images, square_images, triangle_images), axis=0)
y_train = np.concatenate((circle_labels, square_labels, triangle_labels), axis=0)

# Load and preprocess test data
circle_images_test, circle_labels_test = load_images("dataset/Test/circle", 0)
square_images_test, square_labels_test = load_images("dataset/Test/square", 1)
triangle_images_test, triangle_labels_test = load_images("dataset/Test/triangle", 2)

X_test = np.concatenate((circle_images_test, square_images_test, triangle_images_test), axis=0)
y_test = np.concatenate((circle_labels_test, square_labels_test, triangle_labels_test), axis=0)

In [4]:
models = {
    'Linear': make_pipeline(StandardScaler(), svm.SVC(kernel='linear')),
    'Poly': make_pipeline(StandardScaler(), svm.SVC(kernel='poly')),
    'RBF': make_pipeline(StandardScaler(), svm.SVC(kernel='rbf')),
    'Sigmoid': make_pipeline(StandardScaler(), svm.SVC(kernel='sigmoid'))
}

# Train and evaluate each model
for kernel, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy with {kernel} kernel: {accuracy}\n*********************")

Accuracy with Linear kernel: 0.9794871794871794
*********************
Accuracy with Poly kernel: 0.6461538461538462
*********************
Accuracy with RBF kernel: 0.958974358974359
*********************
Accuracy with Sigmoid kernel: 0.9128205128205128
*********************


In [5]:
for kernel, model in models.items():
    param_grid = {'svc__C': [1, 10, 100, 1000], 'svc__kernel': [kernel.lower()]}
    grid = GridSearchCV(model, param_grid, cv=5)
    grid.fit(X_train, y_train)
    
    print(f"Best parameters found for {kernel} kernel: {grid.best_params_}")

    # Use the best model from hyperparameter tuning
    best_model = grid.best_estimator_
    y_pred_best = best_model.predict(X_test)
    accuracy_best = accuracy_score(y_test, y_pred_best)
    print(f"Accuracy with best {kernel} model: {accuracy_best}")
    print("*********************")

Best parameters found for Linear kernel: {'svc__C': 1, 'svc__kernel': 'linear'}
Accuracy with best Linear model: 0.9794871794871794
*********************
Best parameters found for Poly kernel: {'svc__C': 100, 'svc__kernel': 'poly'}
Accuracy with best Poly model: 0.958974358974359
*********************
Best parameters found for RBF kernel: {'svc__C': 1000, 'svc__kernel': 'rbf'}
Accuracy with best RBF model: 0.9948717948717949
*********************
Best parameters found for Sigmoid kernel: {'svc__C': 1, 'svc__kernel': 'sigmoid'}
Accuracy with best Sigmoid model: 0.9128205128205128
*********************
